In [ ]:
import random
import smtplib
from email.mime.text import MIMEText

participants = {}
assignments = {}

def input_participants():
    num_participants = int(input("Enter the number of participants: "))
    for i in range(num_participants):
        name = input(f"Enter the name of participant {i+1}: ")
        email = input(f"Enter {name}'s email: ")
        wishlist = input(f"Enter {name}'s wishlist: ")
        participants[name] = {'email': email, 'wishlist': wishlist}

def input_rules():
    while True:
        name1 = input("Enter the first person's name for a pairing rule (or 'done' to finish): ")
        if name1.lower() == 'done':
            break
        name2 = input(f"Enter the second person's name not to be paired with {name1}: ")
        participants[name1].setdefault('no_pair', []).append(name2)

def generate_assignments():
    potential_assignments = list(participants.keys())
    for name in participants:
        while True:
            assignment = random.choice(potential_assignments)
            if assignment != name and assignment not in assignments and not is_restricted(name, assignment):
                assignments[name] = assignment
                potential_assignments.remove(assignment)
                break

def is_restricted(name, assignment):
    if 'no_pair' in participants[name] and assignment in participants[name]['no_pair']:
        return True
    return False

def send_email(sender_email, sender_password, recipient_email, subject, message):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        msg = MIMEText(message)
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        server.sendmail(sender_email, recipient_email, msg.as_string())
        server.quit()
    except Exception as e:
        print(f"Failed to send email: {e}")

def main():
    input_participants()
    input_rules()
    generate_assignments()

    for name, assignment in assignments.items():
        recipient_email = participants[assignment]['email']
        wishlist = participants[assignment]['wishlist']
        subject = "Your Secret Santa Assignment"
        message = f"Hello {name},\n\nYou are the Secret Santa for {assignment}. Their wishlist is: {wishlist}.\n\nHappy gifting!"
        sender_email = input("Enter your Gmail address: ")
        sender_password = input("Enter your Gmail password: ")  # Be cautious with your password

        send_email(sender_email, sender_password, recipient_email, subject, message)
        print(f"Assignment sent to {name} ({participants[name]['email']})")

if __name__== "__main__":
    main()